In [5]:
import cv2
import numpy as np
from keras.models import load_model
import IPython.display
from PIL import Image

# Load the pre-trained smile/non-smile model
model = load_model('E:/Programming/Projects/GENKI-4K/smile_detection_model_VGG_HF17_v6.h5')

# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def preprocess_image(image):
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, (160, 160))  # Resize to 224x224
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def show_image(image):
    # Convert the image to a format that IPython can display
    image = Image.fromarray(image)
    IPython.display.display(image)
    IPython.display.clear_output(wait=True)

# Initialize the webcam (0 is the default camera)
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video stream from webcam.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # If frame is read correctly, ret is True
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert the frame to grayscale (Haar Cascade works with grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around the faces and predict smile/non-smile
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]  # Crop the face
        preprocessed_face = preprocess_image(face_img)  # Preprocess the face image

        # Predict using the model
        pred = model.predict(preprocessed_face).flatten()

        # Determine the label
        label = "Smile" if pred > 0.5 else "Non-smile"
        confidence = pred[0]

        # Draw rectangle around the face and label it
        color = (0, 255, 0) if label == "Smile" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, f"{label} ({confidence:.2f})", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Display the resulting frame using IPython
    show_image(frame)

# Release the webcam and close windows
cap.release()


KeyboardInterrupt: 